# source_to_raw

In [1]:
# Import packages & setting up a connection to the ENTSOE API Client
import pandas as pd
from entsoe import EntsoePandasClient
client = EntsoePandasClient(api_key = "")
type_marketagreement_type = "A01"
contract_marketagreement_type = "A01"

In [2]:
# Create a Local Reference
import os
current_dir = os.path.abspath("")
parent_dir = os.path.abspath(current_dir + "/../")
print(current_dir)
print(parent_dir)

c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject\1. source_to_raw
c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject


In [3]:
# A dictionary of dates
date_dic = {
    "2017": ["20170101", "20171231"],
    "2018": ["20180101", "20181231"],
    "2019": ["20190101", "20191231"],
    "2020": ["20200101", "20201231"],
    "2021": ["20210101", "20211231"],
    "2022": ["20220101", "20230101"],
    "2023": ["20230101", "20231231"]
}

In [4]:
# A list of timezones
timezone_list = [
    "Europe/Stockholm"
]


In [5]:
# A list of countries
country_list = [
    "SE_1",
    "SE_2",
    "SE_3",
    "SE_4",
    "SE",
    "DE",
    "FR",
    "DK"
]

In [6]:
# Function for extracting Load/Consumption data from entsoe
def load_df_file():
        for value in date_dic.values():
                start_date = value[0]
                end_date = value[1]

                for i in timezone_list:
                        tz = i

                        start_date = pd.Timestamp(start_date, tz = tz)
                        end_date = pd.Timestamp(end_date, tz = tz)

                        for i in country_list:
                                country_code = i

                                load_df = client.query_load_and_forecast(
                                        country_code,
                                        start = start_date,
                                        end = end_date
                                )

                                # Convert RangeIndex to DatetimeIndex & Resample "D"
                                load_df.index = pd.to_datetime(load_df.index)
                                load_df = load_df.resample("D").sum()

                                # Load to .csv
                                load_df.to_csv(
                                        f"{parent_dir}/data/raw/{value[0][0:4]}_{country_code}_load.csv",
                                        # index = False,
                                        date_format = "%Y-%m-%d"
                                )
                                
                                # # Load to .json
                                # load_df.to_json(
                                #         f"{parent_dir}/data/test/{value[0][0:4]}_{country_code}_load.json",
                                #         indent = 4,
                                #         orient = "records",
                                        
                                # )

In [7]:
# # Function for extracting Generation/Production data from entsoe
def generation_df_file():
        for value in date_dic.values():
                start_date = value[0]
                end_date = value[1]

                for i in timezone_list:
                        tz = i

                        start_date = pd.Timestamp(start_date, tz = tz)
                        end_date = pd.Timestamp(end_date, tz = tz)

                        for i in country_list:
                                country_code = i

                                generation_df = client.query_generation(
                                        country_code,
                                        start = start_date,
                                        end = end_date,
                                        psr_type = None
                                )

                                # Convert RangeIndex to DatetimeIndex & Resample "D"
                                generation_df.index = pd.to_datetime(generation_df.index)
                                generation_df = generation_df.resample("D").sum()

                                # Load to .csv
                                generation_df.to_csv(
                                        f"{parent_dir}/data/raw/{value[0][0:4]}_{country_code}_generation.csv",
                                        # index = False,
                                        date_format = "%Y-%m-%d"
                                )
                                
                                # # Load to .json
                                # generation_df.to_json(
                                #         f"{parent_dir}/data/raw/{value[0][0:4]}_{country_code}_generation.json",
                                #         indent = 4,
                                #         orient = "records",
                                # )

In [8]:
load_df_file()

In [9]:
generation_df_file()